In [7]:
from pathlib import Path
import json
import time
import datetime
import os
import pandas as pd
from tqdm.auto import tqdm
from google import genai
from google.genai import types

In [8]:
# --- Configuration ---
SYSTEM_PROMPT = """
You are a **Senior Safety Analyst and Behavioral Scientist** specializing in digital communications. Your mission is to annotate online messages (e.g., social media posts, private messages, forum discussions, chat logs) to build and refine intelligent safety systems. Your expert annotations are critical for training AI models to detect and intervene in real-time situations involving emotional distress, predatory behavior, and self-harm. You must operate with clinical precision, objectivity, and a deep understanding of online communication nuances.

---
### **Core Objective**
To accurately identify and label specific risk categories within digital messages to enable AI-driven safety interventions. Your analysis must be grounded solely in the provided text, without making external assumptions.

---
### **1. Label Set & Definitions**
Assign one or more of the following labels to each message.

* **`Suicidal Reference`**
    * **Definition:** Any explicit or implicit mention of suicide, self-harm, or the desire to end one's life. This includes expressing feelings of being a burden, having a plan, or saying goodbye in a terminal way.
    * **Keywords:** *kill myself, end it all, not be here anymore, tired of living, better off without me, self-harm, cutting.*
    * **Examples:**
        * *"I can't take this pain anymore. I think I'm just going to end it."*
        * *"Everyone would be happier if I wasn't around."*
        * *"Just bought a rope. It's time to say goodbye."*
        * *"Thinking about overdosing again tonight."*

* **`Emotional Distress`**
    * **Definition:** Clear indicators of significant psychological or emotional suffering, such as depression, severe anxiety, hopelessness, or intense emotional pain that is not suicidal in nature.
    * **Keywords:** *depressed, empty, worthless, can't stop crying, panic attack, crippled by anxiety, hate myself, everything is pointless.*
    * **Examples:**
        * *"I just feel so hollow inside, like nothing matters anymore."*
        * *"This overwhelming sadness won't go away and I don't know why."*
        * *"I've been having panic attacks all week and I can't leave my room."*
        * *"I feel completely worthless and alone."*

* **`Attacking Behavior`**
    * **Definition:** Any hostile action directed at another individual or group. This is a broad category that includes threats, bullying, harassment, intimidation, and predatory grooming behavior.
    * **Subcategories to consider:**
        * **Predatory Grooming:** Attempts to build trust with a minor for exploitation, suggesting secrets, or asking for inappropriate content.
        * **Threats:** Explicit or implicit statements of intent to cause harm to others.
        * **Bullying/Harassment:** Persistent insults, mockery, humiliation, or spreading rumors.
        * **Intimidation:** Using fear or power to control someone's actions.
    * **Examples:**
        * *(Grooming)*: *"Your parents wouldn't understand our connection. This is our special secret."*
        * *(Threat)*: *"You better watch your back. I know where you live."*
        * *(Bullying)*: *"Everyone is laughing at you. You're such a loser."*
        * *(Intimidation)*: *"If you tell anyone about this, I'll make sure you regret it."*

* **`Safe/Neutral`**
    * **Definition:** The message shows no signs of the risk categories above. It can be positive, informative, mundane, or emotionally balanced. This label **cannot** be used with any other label.
    * **Examples:**
        * *"Just finished a great workout! Feeling energized."*
        * *"What time is the meeting tomorrow?"*
        * *"Happy birthday! Hope you have an amazing day."*

* **`Unclear`**
    * **Definition:** The message is too ambiguous, lacks critical context, or is too short to make a confident judgment. This is used when the intent cannot be reasonably determined. This label **cannot** be used with any other label.
    * **Examples:**
        * *"I'm done."* (Could be suicidal, angry, or simply finished with a task.)
        * *"Whatever."*
        * *"He'll be sorry."* (Could be a threat or just venting frustration.)

---
### **2. Labeling Instructions & Cognitive Workflow**

Follow this process for every message:

1.  **Initial Read-Through:** Read the message to get a general sense of its content and tone.
2.  **Risk Analysis:** Actively screen for indicators of `Suicidal Reference`, `Emotional Distress`, and `Attacking Behavior`.
    * Does the message mention self-harm or ending a life? -> **`Suicidal Reference`**.
    * Does it express intense sadness, hopelessness, or anxiety without being suicidal? -> **`Emotional Distress`**.
    * Is it directed at another person in a threatening, bullying, or grooming manner? -> **`Attacking Behavior`**.
3. **Labels:** Only use 5 given labels, do not use anything besides Suicidal Reference, Emotional Distress, Attacking Behavior, Safe/Neutral and Unclear
4.  **Contextual Judgment:**
    * A single message may contain multiple risk factors. For example, a message threatening self-harm *in order to manipulate someone* could be labeled both `Suicidal Reference` and `Attacking Behavior`.
    * If no risk indicators are present, label it **`Safe/Neutral`**.
    * If the meaning is genuinely ambiguous after careful consideration and you cannot confidently choose a label, use **`Unclear`**. Do not guess.
5.  **Final Verification:** Ensure your chosen labels strictly adhere to the definitions. Confirm that `Safe/Neutral` or `Unclear` are not combined with other labels.
6.  **Format Output:** Structure your final response **only** in the specified JSON format.

---
### **3. Response Format**

Your response must be a valid JSON object containing a single key, `"labels"`, with its value being a list of strings. The list must contain one or more of the approved labels.

**--- Example Outputs ---**

`{"labels": ["Emotional Distress"]}`

`{"labels": ["Suicidal Reference", "Emotional Distress"]}`

`{"labels": ["Attacking Behavior"]}`

`{"labels": ["Safe/Neutral"]}`

`{"labels": ["Unclear"]}`
"""

In [9]:
INPUT_FILE = Path("Twitter.xlsx")       
GEMINI_MODEL = "gemini-2.5-flash-preview-05-20"  
CHUNK_SIZE = 10
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]  

In [10]:
client = genai.Client(api_key=GEMINI_API_KEY)

def label_text(text: str) -> dict[str, list[str]]:
    text = (text or "").strip()
    if not text:
        return {"labels": ["Unclear - Empty Text"]}
    
    prompt = f"{SYSTEM_PROMPT}\n\nText to classify:\n{text}"
    
    try:
        resp = client.models.generate_content(
            model=GEMINI_MODEL,
            contents=[prompt],
            config=types.GenerateContentConfig(
                response_mime_type="application/json", 
                temperature=0
            ),
        )
        candidate = resp.candidates[0]
        if candidate.finish_reason != "STOP":
            return {"labels": ["Error - Response Blocked"]}
        if candidate.content is None:
            return {"labels": ["Error - No Content"]}
        if not candidate.content.parts or len(candidate.content.parts) == 0:
            return {"labels": ["Error - No Parts"]}
        response_content = candidate.content.parts[0].text
        if response_content is None:
            return {"labels": ["Error - No Text"]}
        
        # Parse JSON
        try:
            result = json.loads(response_content)
            if "labels" not in result:
                return {"labels": ["Error - Invalid Format"]}
            return {"labels": result["labels"]}
        except json.JSONDecodeError as e:
            return {"labels": ["Error - Invalid JSON"]}
            
    except Exception as e:
        print(f"API call failed: {e}")
        return {"labels": ["Error - API Failure"]}

In [11]:
def main() -> None:
    print(f"Loading data from {INPUT_FILE}…")
    try:
        all_sheets = pd.read_excel(INPUT_FILE, sheet_name=None)
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return

    # Gather every "body" column we can find
    bodies = (
        pd.concat(
            [df["body"] for df in all_sheets.values() if "body" in df.columns],
            ignore_index=True
        )
        .dropna()
        .astype(str)
    )

    bodies = bodies[~bodies.str.strip().str.lower().isin(["[removed]", "[deleted]"])]

    # subset_df = bodies.iloc[458020:].reset_index(drop=True).to_frame(name="body")
    # subset_df["labels"] = "" 
    # total_rows = len(subset_df)
    # print(f"Selected {total_rows} messages for labelling.")

    subset_df = bodies.reset_index(drop=True).to_frame(name="body")
    subset_df["labels"] = ""  # placeholder column
    total_rows = len(subset_df)
    print(f"Selected {total_rows} messages for labelling.")

    # Pre-compute filenames
    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    final_outfile = INPUT_FILE.with_name(f"Toxicity_2_{ts}.xlsx")
    progress_file = INPUT_FILE.with_name(f"Toxicity_2_cp_{ts}.xlsx")

    print("Starting labelling process…")
        
    for idx, msg in enumerate(tqdm(subset_df["body"], desc="Labelling", total=total_rows), 1):
        time.sleep(0.2)
                
        # Get labels for the message
        rec = label_text(msg)
        labels_list = rec.get("labels", ["Unclear - Processing Error"])
        subset_df.at[idx - 1, "labels"] = ", ".join(labels_list)

        # Checkpoint save
        if idx % CHUNK_SIZE == 0 or idx == total_rows:
            try:
                subset_df.iloc[:idx].to_excel(progress_file, index=False)
                print(f"✓ Checkpoint: ({idx}/{total_rows}) ➜ {progress_file}")
            except Exception as e:
                print(f"Warning: Could not save checkpoint: {e}")

    # Final save
    try:
        subset_df.to_excel(final_outfile, index=False)
        print(f"✓ Finished. Full file saved ➜ {final_outfile}")
    except Exception as e:
        print(f"Error saving final file: {e}")

In [12]:
if __name__ == "__main__":
    main()

Loading data from Twitter.xlsx…
Selected 3075 messages for labelling.
Starting labelling process…


Labelling:   0%|          | 0/3075 [00:00<?, ?it/s]

✓ Checkpoint: (10/3075) ➜ Toxicity_2_cp_20250612_164618.xlsx
✓ Checkpoint: (20/3075) ➜ Toxicity_2_cp_20250612_164618.xlsx
✓ Checkpoint: (30/3075) ➜ Toxicity_2_cp_20250612_164618.xlsx
✓ Checkpoint: (40/3075) ➜ Toxicity_2_cp_20250612_164618.xlsx
✓ Checkpoint: (50/3075) ➜ Toxicity_2_cp_20250612_164618.xlsx
✓ Checkpoint: (60/3075) ➜ Toxicity_2_cp_20250612_164618.xlsx
✓ Checkpoint: (70/3075) ➜ Toxicity_2_cp_20250612_164618.xlsx
✓ Checkpoint: (80/3075) ➜ Toxicity_2_cp_20250612_164618.xlsx
✓ Checkpoint: (90/3075) ➜ Toxicity_2_cp_20250612_164618.xlsx
✓ Checkpoint: (100/3075) ➜ Toxicity_2_cp_20250612_164618.xlsx
✓ Checkpoint: (110/3075) ➜ Toxicity_2_cp_20250612_164618.xlsx
✓ Checkpoint: (120/3075) ➜ Toxicity_2_cp_20250612_164618.xlsx
✓ Checkpoint: (130/3075) ➜ Toxicity_2_cp_20250612_164618.xlsx
✓ Checkpoint: (140/3075) ➜ Toxicity_2_cp_20250612_164618.xlsx
✓ Checkpoint: (150/3075) ➜ Toxicity_2_cp_20250612_164618.xlsx
✓ Checkpoint: (160/3075) ➜ Toxicity_2_cp_20250612_164618.xlsx
✓ Checkpoint: (17